In [18]:
import os
import platform
import random
import time
import sys

import numpy as np
import scipy as sp
import tensorflow as tf
import pickle

In [19]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling2D, TimeDistributed, Conv1D, Conv2D, GlobalAveragePooling1D, MaxPool2D, Flatten, add
from keras.models import Model
from keras.optimizers import Adam

# silence tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# getting rid of the warning messages about optimizer graph
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)

In [20]:
# print Tensorflow and CUDA information
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
 
if tf.test.gpu_device_name():
    gpu_devices = tf.config.list_physical_devices('GPU')
    details = tf.config.experimental.get_device_details(gpu_devices[0])
    name = details.get('device_name', 'Unknown GPU')
    
    print(f"Using {name}")
else:
    print("No GPU found")

Num GPUs Available:  1
Num CPUs Available:  1
Tensorflow version: 2.11.0
Keras version: 2.11.0
Using NVIDIA A100-SXM4-80GB


In [21]:
import vggish_params as params


path = 'vggish_model.ckpt'

class VGGish(tf.keras.Model):
    def __init__(self, training=False):
        super(VGGish, self).__init__()
        self.training = training

        # The VGG stack of alternating convolutions and max-pools.
        self.conv1 = Conv2D(64, kernel_size=[3, 3], padding='same', activation=tf.nn.relu, trainable=self.training)
        self.pool1 = MaxPool2D(pool_size=[2, 2], padding='same', trainable=self.training)
        self.conv2 = Conv2D(128, kernel_size=[3, 3], padding='same', activation=tf.nn.relu, trainable=self.training)
        self.pool2 = MaxPool2D(pool_size=[2, 2], padding='same', trainable=self.training)
        self.conv3_1 = Conv2D(256, kernel_size=[3, 3], padding='same', activation=tf.nn.relu, trainable=self.training)
        self.conv3_2 = Conv2D(256, kernel_size=[3, 3], padding='same', activation=tf.nn.relu, trainable=self.training)
        self.pool3 = MaxPool2D(pool_size=[2, 2], padding='same', trainable=self.training)
        self.conv4_1 = Conv2D(512, kernel_size=[3, 3], padding='same', activation=tf.nn.relu, trainable=self.training)
        self.conv4_2 = Conv2D(512, kernel_size=[3, 3], padding='same', activation=tf.nn.relu, trainable=self.training)
        self.pool4 = MaxPool2D(pool_size=[2, 2], padding='same', trainable=self.training)

        # Flatten before entering fully-connected layers
        self.flatten = Flatten()
        self.fc1_1 = Dense(4096, activation=tf.nn.relu, trainable=self.training)
        self.fc1_2 = Dense(4096, activation=tf.nn.relu, trainable=self.training)
        # The embedding layer.
        self.fc2 = Dense(params.EMBEDDING_SIZE, activation=None, trainable=self.training)

    def call(self, inputs):
        net = self.conv1(inputs)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3_1(net)
        net = self.conv3_2(net)
        net = self.pool3(net)
        net = self.conv4_1(net)
        net = self.conv4_2(net)
        net = self.pool4(net)

        net = self.flatten(net)
        net = self.fc1_1(net)
        net = self.fc1_2(net)
        net = self.fc2(net)
        
        return net

    def load_vggish_slim_checkpoint(self, checkpoint_path):
        """Loads a pre-trained VGGish-compatible checkpoint."""
        self.load_weights(checkpoint_path)

vggish = VGGish()
vggish.load_vggish_slim_checkpoint(path)


In [22]:
import vggish_input

class VGGishClassifier(tf.keras.Model):
    def __init__(self, vggish_model, num_classes):
        super(VGGishClassifier, self).__init__()
        self.vggish_model = vggish_model
        self.dense1 = Dense(512, activation='relu')
        self.dense2 = Dense(256, activation='relu')
        self.dense3 = Dense(128, activation='relu')
        self.skip1 = Dense(128, activation='relu')
        self.dense4 = Dense(num_classes, activation='sigmoid')
        self.dropout = Dropout(0.5)

    def call(self, inputs):
        x = self.vggish_model(inputs)
        x = self.dense1(x)
        x = self.dropout(x)
        skip = self.skip1(x)
        x = self.dense2(x)
        x = self.dropout(x)
        x = self.dense3(x)
        x = self.dropout(x)
        x = add([x, skip])
        x = self.dense4(x)
        return x



num_classes = 20  # Set the number of classes as needed
classifier_model = VGGishClassifier(vggish, num_classes)

# Compile the model
classifier_model.compile(optimizer=tf.keras.optimizers.Adam(),
                         loss=tf.keras.losses.BinaryCrossentropy(),
                         metrics=['accuracy'])

# Prepare the input data and labels
batch_size = 10
num_frames = params.NUM_FRAMES
num_bands = params.NUM_BANDS

input_data = np.random.rand(batch_size, num_frames, num_bands, 1).astype(np.float32)
# labels = np.random.randint(0, num_classes, size=(batch_size,))
# force all labels to be the same
labels = np.ones((batch_size,)) * 5

classifier_model.build(input_shape=(None, params.NUM_FRAMES, params.NUM_BANDS, 1))
classifier_model.summary()

# Train the classifier model
if False:
    classifier_model.fit(input_data, labels, epochs=10)

    predictions = classifier_model.predict(input_data)

    print(f"Predictions shape: {predictions.shape}")

Model: "vg_gish_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vg_gish_2 (VGGish)          multiple                  72141184  
                                                                 
 dense_9 (Dense)             multiple                  66048     
                                                                 
 dense_10 (Dense)            multiple                  131328    
                                                                 
 dense_11 (Dense)            multiple                  32896     
                                                                 
 dense_12 (Dense)            multiple                  65664     
                                                                 
 dense_13 (Dense)            multiple                  2580      
                                                                 
 dropout (Dropout)           multiple           

In [23]:
import tensorflow as tf
from keras.layers import Dense, Dropout, Add
# import Rescaling
from keras.layers.preprocessing.image_preprocessing import Rescaling
from keras import layers

import tensorflow as tf
from keras.layers import Layer, MultiHeadAttention, Dense, LayerNormalization, Dropout, Reshape, Add

class TransformerBlock(Layer):
    def __init__(self, d_model, num_heads, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(d_model),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    @tf.function
    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

class VGGishTransformerClassifier(tf.keras.Model):
    def __init__(self, vggish_model, num_classes, num_heads=4, ff_dim=512):
        super(VGGishTransformerClassifier, self).__init__()
        self.vggish_model = vggish_model
        self.reshape = Reshape((1, 128))  # Reshaping the output to (batch_size, 1, 128)
        self.transformer_block = TransformerBlock(d_model=128, num_heads=num_heads, ff_dim=ff_dim)
        self.dense1 = Dense(128, activation='relu')
        self.skip1 = Dense(128, activation='relu')
        self.flatten = Flatten()
        self.dense2 = Dense(num_classes, activation='sigmoid')
        self.dropout = Dropout(0.5)

    @tf.function
    def call(self, inputs, training=None):
        x = self.vggish_model(inputs)
        x = self.reshape(x)
        x = self.transformer_block(x)
        x = tf.squeeze(x, axis=1)  # Squeezing the output back to (batch_size, 128)
        x = self.dense1(x)
        x = self.dropout(x)
        skip = self.skip1(x)
        x = Add()([x, skip])
        x = self.flatten(x)
        x = self.dense2(x)
        return x


In [24]:
import os
import pandas as pd
import librosa
import numpy as np

# train csv path
train_csv_path = 'openmic-2018/partitions/split01_train.csv'
# test csv path
test_csv_path = 'openmic-2018/partitions/split01_test.csv'

# open csvs
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# convert to numpy arrays
train_df = train_df.to_numpy()
test_df = test_df.to_numpy()

# make each a single list
train_df = train_df.flatten()
test_df = test_df.flatten()

# print the first 5 rows of the train and test dataframes
print(train_df[:5])
print(test_df[:5])

# only use the first 10% of each csv
train_df = train_df[:int(len(train_df) * 1)]
test_df = test_df[:int(len(test_df) * 1)]

['000135_483840' '000139_119040' '000141_153600' '000144_30720'
 '000145_172800']
['000308_61440' '000312_184320' '000319_145920' '000321_218880'
 '000327_88320']


In [25]:
dataset_path = 'spectrograms'
labels_path = 'labels.csv'

# Read the labels CSV file
# ['filename' 'clarinet' 'flute' 'trumpet' 'saxophone' 'voice' 'accordion' 'ukulele' 'mallet_percussion' 'piano' 'guitar' 'mandolin' 'banjo' 'synthesizer' 'trombone' 'organ' 'drums' 'bass' 'cymbals' 'cello' 'violin']
labels_df = pd.read_csv(labels_path)

# Get the list of all the filenames
filenames = labels_df['filename'].values.tolist()

# load the spectrograms and labels
spectrograms_train = []
labels_train = []

spectrograms_test = []
labels_test = []

#check if pickle file exists
if not os.path.isfile('pickle/spectrograms_train.pkl'):
    
    for filename in filenames:
        # if the filename is not in the train or test dataframe, skip it
        if filename not in train_df and filename not in test_df:
            continue

        # load the spectrogram
        spectrogram = np.load(os.path.join(dataset_path, filename + '.npy'))

        # the fist index is the filename, the next 20 are the labels and the last 20 are the masks
        label = labels_df[labels_df['filename'] == filename].values.tolist()[0][1:21]
        mask = labels_df[labels_df['filename'] == filename].values.tolist()[0][21:]

        # threshold the labels
        label = np.array(label) > 0.5

        # make a pair of the spectrogram and the label
        combined = list(zip(label, mask))

        # append each second seperatly
        if filename in train_df:
            for i in range(10):
                spectrograms_train.append(spectrogram[i])
                labels_train.append(combined)
        elif filename in test_df:
            for i in range(10):
                spectrograms_test.append(spectrogram[i])
                labels_test.append(combined)
        else:
            continue
            # print(f"Filename {filename} not found in train or test dataframes")

    # convert the lists to numpy arrays
    spectrograms_train = np.array(spectrograms_train)
    labels_train = np.array(labels_train)

    spectrograms_test = np.array(spectrograms_test)
    labels_test = np.array(labels_test)

    #spectrograms = spectrograms.reshape(spectrograms.shape[0], num_frames, num_bands, 1)
    spectrograms_test = np.expand_dims(spectrograms_test, axis=-1)
    spectrograms_train = np.expand_dims(spectrograms_train, axis=-1)

    #pickle the spectrogram test and train data
    pickle.dump(spectrograms_test, open('pickle/spectrograms_test.pkl', 'wb'))
    pickle.dump(spectrograms_train, open('pickle/spectrograms_train.pkl', 'wb'))
    #pickle the labels
    pickle.dump(labels_test, open('pickle/labels_test.pkl', 'wb'))
    pickle.dump(labels_train, open('pickle/labels_train.pkl', 'wb'))
else:
    #load the spectrogram test and train data
    spectrograms_test = pickle.load(open('pickle/spectrograms_test.pkl', 'rb'))
    spectrograms_train = pickle.load(open('pickle/spectrograms_train.pkl', 'rb'))
    #load the labels
    labels_test = pickle.load(open('pickle/labels_test.pkl', 'rb'))
    labels_train = pickle.load(open('pickle/labels_train.pkl', 'rb'))



print(f"Spectrograms shape: {spectrograms_train.shape}")
print(f"Labels shape: {labels_train.shape}")

Spectrograms shape: (149140, 96, 64, 1)
Labels shape: (149140, 20, 2)


In [17]:
x = vggish.predict(spectrograms_train[:10])

print(x.shape)

1/1 [==============================] - 0s 400ms/step
(10, 128)


In [26]:
@tf.function
def m_binary_crossentropy(y_true, y_pred):
    # Separate labels and masks from y_true
    labels = y_true[..., 0]
    
    # Compute the binary crossentropy
    bce = tf.keras.losses.BinaryCrossentropy()
    loss = bce(labels, y_pred)
    
    return loss

@tf.function
def BCEp(y_true, y_pred, alpha=1.0, beta=0.0, gamma=-1.0):
    # y_true and y_pred have shape (batch_size, 20)

    # Get the ground truth for each label
    y_true_labels = y_true[..., 0]

    # Get the mask for each label
    y_true_masks = y_true[..., 1]

    # ground truth * log(prediction) + (1 - ground truth) * log(1 - prediction)
    # unreduced_bce_func = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    # bce = unreduced_bce_func(y_true_labels, y_pred)

    bce = y_true_labels * tf.math.log(y_pred) + (1 - y_true_labels) * tf.math.log(1 - y_pred)

    # print shape
    print(f"bce shape: {bce.shape}")

    # mask the loss to zero for labels that are not present
    masked_bce = bce * y_true_masks

    # compute proportion of labels that are present
    present_labels = tf.reduce_sum(y_true_masks, axis=1) / 20.0

    # nomalize the present labels then divide by the number of labels
    normalized_present_labels = (alpha * (present_labels ** gamma) + beta) / 20.0
 

    print(f"normalized_present_labels shape: {normalized_present_labels}")

    # reduce the loss
    loss = tf.reduce_sum(masked_bce, axis=1) * normalized_present_labels

    return loss


@tf.function
def m_accuracy(y_true, y_pred):
    # Separate labels and masks from y_true. The masks should be ignored.
    labels = y_true[..., 0]
    masks = y_true[..., 1]

    # Threshold predictions to convert them to binary values (assuming 0.5 as the threshold)
    binary_pred = tf.cast(tf.greater_equal(y_pred, 0.5), tf.float32)

    # Compute the element-wise equality between binary_pred and y_true
    correct_predictions = tf.cast(tf.equal(binary_pred, labels), tf.float32)

    # multiply by the mask
    correct_predictions = correct_predictions * masks

    # Calculate the accuracy
    accuracy = tf.reduce_sum(correct_predictions) / tf.reduce_sum(masks)

    return accuracy

@tf.function
def m_f1(y_true, y_pred):
    # Separate labels and masks from y_true
    labels = y_true[..., 0]
    masks = y_true[..., 1]

    # Threshold predictions to convert them to binary values (assuming 0.5 as the threshold)
    binary_pred = tf.cast(tf.greater_equal(y_pred, 0.5), tf.float32)

    # Calculate true positives, false positives, and false negatives
    true_positives = tf.reduce_sum(binary_pred * labels * masks)
    false_positives = tf.reduce_sum(binary_pred * (1 - labels) * masks)
    false_negatives = tf.reduce_sum((1 - binary_pred) * labels * masks)

    # Calculate precision and recall
    precision = true_positives / (true_positives + false_positives + 1e-8)
    recall = true_positives / (true_positives + false_negatives + 1e-8)

    # Calculate the F1 score
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-8)

    return f1_score

X_train = spectrograms_train
y_train = labels_train

X_test = spectrograms_test
y_test = labels_test

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (149140, 96, 64, 1)
y_train shape: (149140, 20, 2)
X_test shape: (50840, 96, 64, 1)
y_test shape: (50840, 20, 2)


In [27]:
# vgg_X_train = vggish.predict(X_train)
# vgg_X_test = vggish.predict(X_test)

# print(f"vgg_X_train shape: {vgg_X_train.shape}")
# print(f"vgg_X_test shape: {vgg_X_test.shape}")

1589/1589 [==============================] - 6s 4ms/step
vgg_X_train shape: (149140, 128)
vgg_X_test shape: (50840, 128)


In [28]:
#pickle the vgg_X_test and vgg_X_train data
# pickle.dump(vgg_X_test, open('pickle/vgg_X_test.pkl', 'wb'))
# pickle.dump(vgg_X_train, open('pickle/vgg_X_train.pkl', 'wb'))

#load the vgg_X_test and vgg_X_train data
vgg_X_test = pickle.load(open('pickle/vgg_X_test.pkl', 'rb'))
vgg_X_train = pickle.load(open('pickle/vgg_X_train.pkl', 'rb'))
print(f"vgg_X_train shape: {vgg_X_train.shape}")
print(f"vgg_X_test shape: {vgg_X_test.shape}")

In [30]:
print (vgg_X_train[0])

[ 0.03333668  0.01239415  0.02070752  0.02415475 -0.0193878  -0.00188267
  0.03266878 -0.03517116  0.00617049 -0.01298855  0.03187737 -0.03661109
 -0.02493214  0.03001267  0.07322174 -0.03755243 -0.00109528  0.01354072
  0.01612341 -0.03816056 -0.0046137  -0.05460477  0.02413895  0.05334652
 -0.00412781 -0.01510066  0.00227696 -0.01563813 -0.04267376  0.04319832
 -0.02992732 -0.00603651  0.02305216 -0.05009295  0.02032151 -0.02937008
  0.00359633 -0.01117912 -0.02908186 -0.01970048 -0.00572515  0.03168309
 -0.01354233 -0.05525997 -0.03986001  0.01863924 -0.01167305 -0.01035044
  0.00624337 -0.0528811   0.03228129 -0.02697513 -0.01131375 -0.00473073
 -0.01244172 -0.02195379 -0.00179833 -0.00452174 -0.03076958  0.04686664
 -0.00531481  0.00782339 -0.0277487  -0.01618263 -0.02501273  0.00779569
  0.02077115 -0.00990653  0.02501084  0.00406783  0.00570872 -0.02781973
 -0.00594668 -0.02384339 -0.03762325 -0.00755598  0.03127922 -0.00738188
  0.02148581 -0.01868164  0.03107121 -0.03537071  0

In [41]:
#reshape the data such that it can be fed into the model in this format (samples_num, freq_bins, time_steps, 1), where each sample is 10 seconds long
vgg_X_train_tensor = vgg_X_train.reshape((int(vgg_X_train.shape[0]/10),10, vgg_X_train.shape[1]))
vgg_X_test_tensor = vgg_X_test.reshape((int(vgg_X_test.shape[0]/10),10, vgg_X_test.shape[1]))


In [42]:
print(f"vgg_X_train_tensor shape: {vgg_X_train_tensor.shape}")
print(f"vgg_X_test_tensor shape: {vgg_X_test_tensor.shape}")

vgg_X_train_tensor shape: (14914, 10, 128)
vgg_X_test_tensor shape: (5084, 10, 128)


In [31]:
#load the pytorch model
import torch
model = torch.load('model_0.pt')

In [44]:
#evalueate the model on the first 10 samples in the training set
device = torch.device('cuda')
model.to(device)

model.eval()
with torch.no_grad():
    y_pred = model(torch.from_numpy(vgg_X_train_tensor[0:10]).float().to(device))
    print(y_pred.shape)

torch.Size([10, 20])


In [ ]:
class UNet:
    def __init__(self, input_shape, num_filters):
        self.input_shape = input_shape
        self.num_filters = num_filters
        self.classifier = model
        self.embedding = vggish
        self.build()

    def conv_block(self, input, num_filters):
        x = Conv2D(num_filters, 3, padding="same", activation="relu")(input)
        x = BatchNormalization()(x)
        x = Conv2D(num_filters, 3, padding="same", activation="relu")(x)
        x = BatchNormalization()(x)
        return x

    def encoder_block(self, input, num_filters):
        x = self.conv_block(input, num_filters)
        p = MaxPooling2D((2, 2))(x)
        return x, p

    def decoder_block(self, input, skip_features, num_filters):
        x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
        x = concatenate([x, skip_features], axis=3)
        x = self.conv_block(x, num_filters)
        return x

    def build(self):

        #


        # Input
        inputs = Input(shape=self.input_shape)

        # Encoder
        e1, p1 = self.encoder_block(inputs, self.num_filters)
        e2, p2 = self.encoder_block(p1, self.num_filters*2)
        e3, p3 = self.encoder_block(p2, self.num_filters*4)
        e4, p4 = self.encoder_block(p3, self.num_filters*8)

        # Bridge
        b1 = self.conv_block(p4, self.num_filters*16)

        # Decoder
        d1 = self.decoder_block(b1, e4, self.num_filters*8)
        d2 = self.decoder_block(d1, e3, self.num_filters*4)
        d3 = self.decoder_block(d2, e2, self.num_filters*2)
        d4 = self.decoder_block(d3, e1, self.num_filters)

        # Output
        outputs = Conv2D(1, (1, 1), activation='sigmoid')(d4)

        # Model
        self.model = Model(inputs, outputs)

    def summary(self):
        self.model.summary()

    def compile(self, optimizer, loss, metrics=None):
        self.model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


In [10]:
# import f1
from tensorflow.keras import backend as K

num_classes = 20  # Set the number of classes as needed
# classifier_model = VGGishClassifier(vggish, num_classes)
classifier_model = VGGishTransformerClassifier(vggish, num_classes)

# Compile the model
classifier_model.compile(optimizer=tf.keras.optimizers.Adam(),
                         loss=m_binary_crossentropy,
                         metrics=[m_accuracy, m_f1])

classifier_model.build(input_shape=(None, params.NUM_FRAMES, params.NUM_BANDS, 1))

classifier_model.summary()

history = classifier_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=32, batch_size=128)

predictions = classifier_model.predict(X_test)
print(f"Predictions shape: {predictions.shape}")

# print the first prediction rounded to 2 decimal places
print(f"First prediction: {np.round(predictions[0], 2)}")
print(f"First label:      {y_test[0]}")


Model: "vg_gish_transformer_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vg_gish (VGGish)            multiple                  72141184  
                                                                 
 reshape (Reshape)           multiple                  0         
                                                                 
 transformer_block (Transfor  multiple                 396032    
 merBlock)                                                       
                                                                 
 dense_10 (Dense)            multiple                  16512     
                                                                 
 dense_11 (Dense)            multiple                  16512     
                                                                 
 flatten_1 (Flatten)         multiple                  0         
                                    

InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_21' defined at (most recent call last):
    File "/users/emann/.conda/envs/podElias/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/users/emann/.conda/envs/podElias/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/users/emann/.conda/envs/podElias/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/users/emann/.conda/envs/podElias/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/users/emann/.conda/envs/podElias/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/users/emann/.conda/envs/podElias/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
   